In [1]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")


Libraries imported successfully......


# i. Player data Current Season


In [2]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 447  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


https://fantasy.premierleague.com/api/element-summary/447/


In [3]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()
player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])



In [4]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])

In [5]:
elements = elements_df.loc[:,['id','team','web_name','first_name','second_name']]
elements = elements.rename(columns = {'id': 'player_id', 'team':'team_id'})

In [6]:
# List of columns to drop
columns_to_drop = ['was_home', 'bps', 'kickoff_time','influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded', 'value',
       'transfers_balance', 'selected', 'transfers_in', 'transfers_out','fixture', 'team_h_score',
       'team_a_score']

# Use the drop method to remove the specified columns
player_history_df.drop(columns=columns_to_drop, inplace=True)


In [7]:
player = player_history_df.rename(columns = {'element': 'player_id', 'round': 'event_id'})


In [8]:
filtered_teams = pd.read_csv('filtered_teams.csv')

In [9]:
Haaland = elements[elements['second_name']=='Haaland']

In [10]:
Haaland

,player_id,team_id,web_name,first_name,second_name
447,355,13,Haaland,Erling,Haaland


In [11]:
player =  player.merge(elements, on = 'player_id')
player = player.merge(filtered_teams, on = ['team_id','event_id'])
#player = player.drop_duplicates(subset='event_id')


# ii. Analysis

In [12]:
filtered_teams.columns

Index(['team_id', 'team_name', 'team_short_name', 'unavailable', 'event_id',
       'fixture_id', 'fixture_difficulty', 'is_home', 'kickoff_time',
       'started', 'Attack_Strength', 'team_score', 'Form', 'Numerical_Form',
       'Win_percentage', 'Strength', 'Defence_Strength', 'Home_Form',
       'Home_Numerical_Form', 'Home_Win_percentage', 'Home_Strength',
       'Home_Defence_Strength', 'Home_Attack_Strength', 'Away_Form',
       'Away_Numerical_Form', 'Away_Win_percentage', 'Away_Strength',
       'Away_Defence_Strength', 'Away_Attack_Strength'],
      dtype='object')

In [13]:
player

,player_id,opponent_team,total_points,event_id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,...,Home_Win_percentage,Home_Strength,Home_Defence_Strength,Home_Attack_Strength,Away_Form,Away_Numerical_Form,Away_Win_percentage,Away_Strength,Away_Defence_Strength,Away_Attack_Strength
0,308,7,5,1,76,0,1,0,1,0,...,NaN,NaN,NaN,NaN,D,0.512967,0.000000,0.170941,0.184439,0.272977
1,308,3,5,2,85,1,0,0,1,0,...,100.0,0.874929,0.873992,0.783229,NaN,NaN,NaN,NaN,NaN,NaN
2,308,15,5,3,90,0,1,0,1,0,...,NaN,NaN,NaN,NaN,DW,0.673846,50.000000,0.567725,0.571423,0.609901
3,308,2,8,4,90,1,0,1,0,0,...,100.0,0.927800,0.929055,0.871649,NaN,NaN,NaN,NaN,NaN,NaN
4,308,20,10,5,90,0,2,0,1,0,...,NaN,NaN,NaN,NaN,DWW,0.766154,66.666667,0.697768,0.697946,0.720376
5,308,19,10,6,90,1,0,0,1,0,...,100.0,0.960773,0.961943,0.930595,NaN,NaN,NaN,NaN,NaN,NaN
6,308,18,1,7,73,0,0,0,1,0,...,NaN,NaN,NaN,NaN,DWWL,0.522198,50.000000,0.515735,0.513070,0.530827
7,308,5,15,8,90,2,0,0,2,0,...,NaN,NaN,NaN,NaN,DWWLD,0.520352,40.000000,0.446777,0.446035,0.481458


In [14]:
#Features_engineered = ['Minutes_per_game', 'Player_Strength', 'XA', 'XG', 'XS']
features = ['fixture_difficulty','kickoff_time', 'started',
       'is_home', 'Attack_Strength', 'team_score', 'Form', 'Numerical_Form',
       'Win_percentage', 'Strength', 'Defence_Strength', 'Home_Form',
       'Home_Numerical_Form', 'Home_Win_percentage', 'Home_Strength',
       'Home_Defence_Strength', 'Home_Attack_Strength', 'Away_Form',
       'Away_Numerical_Form', 'Away_Win_percentage', 'Away_Strength',
       'Away_Defence_Strength', 'Away_Attack_Strength']
labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

In [15]:
player['kickoff_time'] = pd.to_datetime(player['kickoff_time'])




In [16]:
player.set_index('kickoff_time', inplace=True)


In [27]:
player = player[player['started']==True]
player['is_home'] = player['is_home'].astype(int)


In [26]:
player.dtypes

player_id                  int64
opponent_team              int64
total_points               int64
event_id                   int64
minutes                    int64
goals_scored               int64
assists                    int64
clean_sheets               int64
goals_conceded             int64
own_goals                  int64
penalties_saved            int64
penalties_missed           int64
yellow_cards               int64
red_cards                  int64
saves                      int64
bonus                      int64
team_id                    int64
web_name                  object
first_name                object
second_name               object
team_name                 object
team_short_name           object
unavailable                 bool
fixture_id               float64
fixture_difficulty       float64
is_home                   object
started                   object
Attack_Strength          float64
team_score               float64
Form                      object
Numerical_

In [28]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Define a custom transformer for feature selection
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.features]

# Load your data into a Pandas DataFrame (Assuming you have already processed the data)
# Feature engineering code is the same as in the previous response

# Define features and labels
features = ['fixture_difficulty', 'is_home', 'Attack_Strength', 'team_score', 'Numerical_Form', 'Win_percentage', 'Strength', 'Defence_Strength']
labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

# Choose a label to predict
label_to_predict = 'goals_scored'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(player[features], player[label_to_predict], test_size=0.2, random_state=42)

# Define a scikit-learn pipeline
pipeline = Pipeline([
    ('feature_selector', FeatureSelector(features)),
    ('scaler', StandardScaler()),
    ('model', tf.keras.Sequential([
        tf.keras.layers.Input(shape=(len(features),)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer with 1 neuron for regression
    ]))
])

# Compile the model within the pipeline
pipeline.named_steps['model'].compile(optimizer='adam', loss='mean_squared_error')

# Train the model within the pipeline
pipeline.named_steps['model'].fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model within the pipeline
mse = pipeline.named_steps['model'].evaluate(X_test, y_test)
print(f"Mean Squared Error on the test set: {mse:.2f}")

# Make predictions within the pipeline
predictions = pipeline.named_steps['model'].predict(X_test)

# You can use the predictions for further analysis or visualization


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 56.1167
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 41.2403
Epoch 3/100
1/1 [==============================] - 0s 11ms/step - loss: 28.6010
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 18.4706
Epoch 5/100
1/1 [==============================] - 0s 12ms/step - loss: 10.8273
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 5.4751
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 2.2089
Epoch 8/100
1/1 [==============================] - 0s 47ms/step - loss: 0.7420
Epoch 9/100
1/1 [==============================] - 0s 15ms/step - loss: 0.7060
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 1.6378
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - loss: 2.9248
Epoch 12/100
1/1 [==============================] - 0s 12ms/step - loss: 4.1982
Epoch 13/100
1/1 [============================

In [29]:
predictions

array([[0.45564684],
       [0.8080485 ]], dtype=float32)